In [1]:
import pandas as pd
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',100)
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline


In [18]:
def print_full(x):
    pd.set_option('display.max_rows',len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [14]:
data_loc = 'D:\\job_hunting\\blend'
 
df = pd.read_csv(data_loc +'\\'+ 'LoanStats3d.csv')

C:\Users\Chao Cheng\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Is Lending Club making money with the loans in the dataset?

### Let's first look at Lending Club business model:
#### Revenue =
     "Lending Club makes money through origination and service fees. Borrowers pay a one-time origination fee of 1.11% to 5% of the total loan amount, depending on the loan grade and term. Meanwhile, investors pay a service fee of 1% of each payment received from a borrower." 
Read more at http://vator.tv/news/2014-03-21-how-does-lending-club-make-money#yss2rMyUWuVUhgqZ.99

In [15]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68615169,73517949,16000,16000,16000.0,60 months,8.49%,328.19,B,B1,Supervisor,5 years,MORTGAGE,62000.0,Source Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,936xx,CA,28.92,0,Mar-2002,0,39.0,NaN,10,0,15763,51.2%,24,w,13083.35,13083.35,4247.60,4247.60,2916.65,1330.95,0.0,0.0,0.0,Feb-2017,328.19,Mar-2017,Feb-2017,0,61.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,227708,3.0,3.0,1.0,2.0,3.0,48608.0,82.0,2.0,4.0,4727.0,71.0,30800,1.0,1.0,0.0,6,22771,8703.0,63.0,0,0,165.0,159,3,3,3,3.0,NaN,17.0,39.0,1,4,5,4,7,9,6,12.0,6,10,0.0,0,0,3,87.5,50.0,0,0,307343,64371,23500,59526
1,68537564,73427399,8000,8000,8000.0,36 months,10.78%,261.08,B,B4,Counselor,< 1 year,MORTGAGE,45000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,482xx,MI,21.23,0,Jan-1998,0,34.0,NaN,28,0,11141,35.5%,69,w,5403.18,5403.18,3382.06,3382.06,2596.82,785.24,0.0,0.0,0.0,Feb-2017,261.08,Mar-2017,Feb-2017,0,61.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,148154,2.0,2.0,0.0,1.0,13.0,136887.0,86.0,4.0,6.0,1984.0,49.0,31400,2.0,2.0,0.0,7,5926,1188.0,79.5,0,0,147.0,215,4,4,2,13.0,NaN,13.0,34.0,1,3,11,3,6,21,25,45.0,12,27,0.0,0,0,4,97.1,100.0,0,0,168819,148154,5800,136996
2,68536799,73426617,10000,10000,10000.0,36 months,10.78%,326.35,B,B4,Lpn,3 years,MORTGAGE,41600.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,472xx,IN,15.78,0,Oct-2004,0,NaN,NaN,18,0,12958,52.5%,33,w,6753.97,6753.97,4227.58,4227.58,3246.03,981.55,0.0,0.0,0.0,Feb-2017,326.35,Mar-2017,Feb-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,168304,2.0,9.0,2.0,3.0,2.0,27723.0,94.0,0.0,2.0,4958.0,75.0,24700,2.0,4.0,2.0,5,9350,4617.0,66.8,0,0,134.0,124,16,2,2,16.0,NaN,7.0,NaN,0,2,5,3,4,16,8,15.0,5,18,0.0,0,0,2,100.0,33.3,0,0,190694,40681,13900,29512
3,68355089,73244858,24700,24700,24700.0,36 months,11.99%,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1,Dec-1999,4,6.0,NaN,22,0,21470,19.2%,38,w,0.00,0.00,25679.66,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,204396,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800,0.0,0.0,6.0,4,9733,57830.0,27.1,0,0,113.0,192,2,2,4,2.0,NaN,0.0,6.0,0,5,5,13,17,

In [16]:
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,4.210950e+05,4.210950e+05,421095.000000,421095.000000,421095.000000,421095.000000,4.210950e+05,421095.000000,421095.000000,421095.000000,217133.000000,74415.000000,421095.000000,421095.000000,4.210950e+05,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,122729.000000,421095.0,511.000000,509.000000,421095.000000,421095.000000,4.210950e+05,21372.000000,21372.000000,21372.000000,21372.000000,20810.000000,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,4.210950e+05,21372.000000,21372.000000,21372.000000,421095.00000,421095.000000,417132.000000,416868.000000,421095.000000,421095.000000,408841.000000,421095.000000,421095.000000,421095.000000,421095.000000,417297.000000,108600.000000,376496.000000,151737.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421095.000000,421094.000000,421095.000000,421095.000000,401865.000000,421095.000000,421095.000000,421095.000000,421095.000000,416856.000000,421095.000000,421095.000000,4.210950e+05,4.210950e+05,421095.000000,4.210950e+05
mean,5.393097e+07,5.759576e+07,15240.285862,15240.285862,15234.156412,441.859660,7.696561e+04,19.195989,0.347397,0.572690,34.023391,66.592609,11.948643,0.233776,1.768548e+04,25.493468,6731.852248,6728.957444,9724.272952,9720.600967,7468.877666,2193.412444,0.573743,61.409098,10.793400,2771.671503,0.020240,45.114741,1.0,109981.011585,18.310118,0.006087,257.478241,1.400755e+05,1.109021,2.928832,0.761651,1.674574,20.912686,36552.811389,71.580491,1.389060,2.975482,5887.979740,60.881995,3.365771e+04,0.943945,1.537058,2.234091,4.66500,13104.784721,9636.063992,62.228136,0.010420,13.765184,127.567419,186.586697,13.451247,8.013204,1.685261,24.856534,39.838177,6.751163,35.797142,0.527054,3.740966,5.860832,4.799879,8.137138,8.649001,8.431786,14.883601,5.809238,11.894537,0.000761,0.004042,0.094318,2.170947,93.924313,47.527110,0.133188,0.064556,1.734070e+05,5.222199e+04,21958.208958,4.347865e+04
std,9.475423e+06,1.013358e+07,8571.325901,8571.325901,8567.935752,244.850838,7.394996e+04,23.504038,0.927695,0.867776,21.990270,25.577899,5.633119,0.649778,2.419503e+04,12.097200,6975.864148,6972.541062,7161.590912,7159.162377,6523.441530,1900.836729,5.317043,461.711020,81.024136,5741.807514,0.157506,22.537808,0.0,52730.379847,7.169233,0.086482,2240.648162,1.552696e+05,1.242675,3.089987,0.996035,1.688725,27.209081,43103.833619,23.021964,1.520129,2.631886,5284.701239,20.013842,3.766404e+04,1.446872,2.723339,2.480242,3.19538,15671.803897,14774.166134,27.576378,0.117019,781.821107,52.075471,95.374931,17.156985,9.189312,1.990160,32.272793,23.003

In [19]:
print_full(df.dtypes)

id                                  int64
member_id                           int64
loan_amnt                           int64
funded_amnt                         int64
funded_amnt_inv                   float64
term                               object
int_rate                           object
installment                       float64
grade                              object
sub_grade                          object
emp_title                          object
emp_length                         object
home_ownership                     object
annual_inc                        float64
verification_status                object
issue_d                            object
loan_status                        object
pymnt_plan                         object
url                                object
desc                               object
purpose                            object
title                              object
zip_code                           object
addr_state                        

In [25]:
pd.Series(df['loan_status']).value_counts()

Current               277527
Fully Paid             94968
Charged Off            33416
Late (31-120 days)      8626
In Grace Period         4674
Late (16-30 days)       1624
Default                  260
Name: loan_status, dtype: int64